In [9]:
# Import necessary libraries
import requests
import pandas as pd
import time

# Set up your RapidAPI credentials
RAPIDAPI_KEY = '206ed0c238msh46a8d350a8fc8e0p19b601jsn99c5cf9a6b96'  # Replace with your actual RapidAPI key
HEADERS = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': RAPIDAPI_KEY
}

# API endpoint to fetch players
API_FOOTBALL_PLAYERS_ENDPOINT = "https://api-football-v1.p.rapidapi.com/v3/players"

# Function to fetch player statistics for a specific season and league (EPL)
def fetch_players(season=2023, league=39):  # 39 is the league ID for EPL
    all_players_data = []
    page = 1
    while True:
        querystring = {
            "league": league,
            "season": season,
            "page": page
        }
        response = requests.get(API_FOOTBALL_PLAYERS_ENDPOINT, headers=HEADERS, params=querystring)
        
        # Check for rate limit errors and adjust
        if response.status_code == 429:
            print("Rate limit exceeded. Pausing to avoid further requests...")
            time.sleep(60)  # Adjust sleep time based on API plan rate limits
            continue  # Retry the current page after the sleep period

        if response.status_code == 200:
            data = response.json()
            players = data['response']
            if not players:
                break  # Exit loop if no more players are found
            
            # Extract relevant player data
            for player_info in players:
                player = player_info['player']
                statistics = player_info['statistics'][0]  # Assuming first set of statistics
                
                # Extract data safely with .get() to avoid KeyErrors
                match_data = {
                    'player_id': player['id'],
                    'player_name': player['name'],
                    'age': player['age'],
                    'nationality': player['nationality'],
                    'team': statistics['team']['name'],
                    'position': statistics['games']['position'],
                    'minutes_played': statistics['games'].get('minutes', 0),
                    'goals_scored': statistics['goals'].get('total', 0),
                    'assists': statistics['goals'].get('assists', 0),
                    'clean_sheets': statistics['games'].get('clean_sheets', 0),  # Adjusted to check available keys
                    'yellow_cards': statistics['cards'].get('yellow', 0),
                    'red_cards': statistics['cards'].get('red', 0),
                    'total_points': statistics['games'].get('rating', None),  # Rating as a proxy for points
                }
                all_players_data.append(match_data)
            
            page += 1  # Go to the next page of results
            time.sleep(5)  # Add a delay to avoid hitting rate limits
        else:
            print(f"Error fetching data: {response.status_code}, {response.text}")
            break
    
    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(all_players_data)
    df.to_csv('data/processed/historical_players_data.csv', index=False)
    print(df.head())

# Run the data fetching function
fetch_players()


OSError: Cannot save file into a non-existent directory: 'data/processed'